In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import requests
import os

## ***Build Weather Tools using realtime api***

In [4]:
load_dotenv()
weather_api = os.getenv("WEATHER_API")
groq_api_key = os.getenv("GROQ_API_KEY")

In [5]:
llm = ChatGroq(model_name = "meta-llama/llama-4-scout-17b-16e-instruct", groq_api_key=groq_api_key)

## *Define the tools*

In [6]:
@tool
def get_weather_data(city: str) -> str:
    """
    This function fetches the current weather data for a given city
    """
    url = f"http://api.weatherstack.com/current?access_key={weather_api}&query={city}"
    response = requests.get(url)
    return response.json()

In [7]:
get_weather_data.invoke(
    {
        "city": "Dhaka"
    }
)

{'request': {'type': 'City',
  'query': 'Dhaka, Bangladesh',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Dhaka',
  'country': 'Bangladesh',
  'region': '',
  'lat': '23.723',
  'lon': '90.409',
  'timezone_id': 'Asia/Dhaka',
  'localtime': '2025-05-03 11:54',
  'localtime_epoch': 1746273240,
  'utc_offset': '6.0'},
 'current': {'observation_time': '05:54 AM',
  'temperature': 33,
  'weather_code': 116,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'],
  'weather_descriptions': ['Partly Cloudy '],
  'astro': {'sunrise': '05:23 AM',
   'sunset': '06:27 PM',
   'moonrise': '10:21 AM',
   'moonset': 'No moonset',
   'moon_phase': 'Waxing Crescent',
   'moon_illumination': 34},
  'air_quality': {'co': '797.35',
   'no2': '59.94',
   'o3': '67',
   'so2': '43.66',
   'pm2_5': '145.595',
   'pm10': '152.995',
   'us-epa-index': '4',
   'gb-defra-index': '4'},
  'wind_speed': 5,
  'wind_degree': 280,
  'wind_dir'

In [10]:
search_tool = TavilySearchResults()

## Now time to define the Agents

In [8]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [9]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [11]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [12]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

## Now time to Test my simple Agent

In [13]:
response = agent_executor.invoke(
    {
        "input": "Find the capital of Bangladesh, then find the its current weather condition today."
    }
)
print(response)



> Entering new AgentExecutor chain...
To find the capital of Bangladesh and then its current weather condition, I should first search for the capital of Bangladesh.

Action: tavily_search_results_json
Action Input: "What is the capital of Bangladesh?"[{'url': 'https://testbook.com/question-answer/what-is-the-capital-of-bangladesh--599c287be07ce02427368b9d', 'content': 'The correct answer is Dhaka. Dhaka is the capital and largest city of Bangladesh. It is the ninth-largest and the sixth-most densely populated city in the'}, {'url': 'https://en.wikipedia.org/wiki/Bangladesh', 'content': 'Emblem\nAnthem:\xa0আমার সোনার বাংলা (Bengali)\nAmar Sonar Bangla\n"My Golden Bengal"\nDuration: 2 minutes and 15 seconds.\n2:15\nGovernment Seal\nShow globe\nShow map of Bangladesh\nShow all\nCapital\nand largest city\n    Dhaka\n23°45′50″N 90°23′20″E\nOfficial language\nand national language   Bengali[1][2]\nRecognised foreign language English[3]\nEthnic\xa0groups (2022 census)[4]  99% Bengali\nshow\

In [14]:
response['output']

'The capital of Bangladesh is Dhaka. The current weather condition in Dhaka is patchy rain nearby with a temperature of 34 degrees Celsius.'